# Notebook for the experiments
In this notebook are contained the following features:
* GRAFF + Link prediction,

The main tools that have been exploited are [PyTorch](https://pytorch.org/) (1.13.0), [PyTorch-Lightning](https://www.pytorchlightning.ai/index.html) (1.5.10), [Pytorch-geometric](https://pytorch-geometric.readthedocs.io/en/latest/install/installation.html) (2.3.0) and [Weights & Biases](https://wandb.ai/)

### Requirements to run the notebook

In [1]:
# !pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
# !pip install pytorch-lightning==1.5.10
# !pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-1.12.0+cu113.html
# !pip install torch_geometric
# !pip install wandb
# !pip install ogb

## Importing the libraries

In [2]:
######## IMPORT EXTERNAL FILES ###########
import torch
import torch.nn.functional as F
import torch.nn.utils.parametrize as parametrize
import torch.nn as nn

import torch_geometric
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.utils import negative_sampling

import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.loggers import WandbLogger

import wandb
######### IMPORT INTERNAL FILES ###########
import sys
sys.path.append("../../src")
from GRAFF import *
from config import *

/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Link prediction features initialized.....


### System configuration

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_gpus = 1 if device == 'cuda' else 0

if wb:
    wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: difra00 (deepl_wizards). Use `wandb login --relogin` to force relogin


## PyTorch Lightning DataModule (Link Prediction)

In [4]:
class DataModuleLP(pl.LightningDataModule):

    def __init__(self,  train_set, val_set, test_set, mode, batch_size):

        self.mode = mode  # "hp" or "test"
        self.batch_size = batch_size
        self.train_set, self.val_set, self.test_set = train_set, val_set, test_set

    def setup(self, stage=None):
        if stage == 'fit':

            # edge_index are the message passing edges,
            # edge_label_index are the supervision edges.
            if self.train_set.pos_edge_label_index.shape[1] < self.train_set.edge_index.shape[1]:
                pos_mask_edge = self.train_set.pos_edge_label_index.shape[1]

                self.train_set.edge_index = self.train_set.edge_index[:,
                                                                      pos_mask_edge:]
            else:
                self.train_set.pos_edge_label_index = self.train_set.edge_index[:,
                                                                                :self.train_set.edge_index.shape[1] // 2]
                self.train_set.neg_edge_label_index = self.train_set.neg_edge_label_index[
                    :, :self.train_set.edge_index.shape[1] // 2]

                self.train_set.edge_index = self.train_set.edge_index[:,
                                                                      self.train_set.edge_index.shape[1] // 2:]

    def train_dataloader(self, *args, **kwargs):
        return DataLoader([self.train_set], batch_size=batch_size, shuffle=False)

    def val_dataloader(self, *args, **kwargs):
        if self.mode == 'hp':
            return DataLoader([self.val_set], batch_size=batch_size, shuffle=False)
        elif self.mode == 'test':
            return DataLoader([self.test_set], batch_size=batch_size, shuffle=False)

In [5]:
train_data = torch.load(dataset_name + "/train_data.pt")
val_data = torch.load(dataset_name + "/val_data.pt")
test_data = torch.load(dataset_name + "/test_data.pt")

In [6]:
# print(train_data)
# print(val_data)
# print(test_data)


In [7]:
mode = 'hp'  # hp: Hyperparameter selection mode
sweep = True
dataM = DataModuleLP(train_data.clone(), val_data.clone(), test_data.clone(), mode = mode, batch_size = batch_size)
dataM.setup(stage='fit')
dataM.setup(stage='test') 


In [8]:
print(dataM.train_set)
print(dataM.val_set)
print(dataM.test_set)


Data(x=[2708, 1433], edge_index=[2, 4224], y=[2708], train_mask=[2708, 10], val_mask=[2708, 10], test_mask=[2708, 10], pos_edge_label=[4224], pos_edge_label_index=[2, 4224], neg_edge_label=[844800], neg_edge_label_index=[2, 844800])
Data(x=[2708, 1433], edge_index=[2, 8448], y=[2708], train_mask=[2708, 10], val_mask=[2708, 10], test_mask=[2708, 10], pos_edge_label=[527], pos_edge_label_index=[2, 527], neg_edge_label=[105400], neg_edge_label_index=[2, 105400])
Data(x=[2708, 1433], edge_index=[2, 9502], y=[2708], train_mask=[2708, 10], val_mask=[2708, 10], test_mask=[2708, 10], pos_edge_label=[527], pos_edge_label_index=[2, 527], neg_edge_label=[105400], neg_edge_label_index=[2, 105400])


### PyTorch Lightning Callbacks

In [9]:

class Get_Metrics(Callback):

    def on_train_epoch_end(self, trainer: "pl.Trainer", pl_module: "pl.LightningModule"):

        # Compute the metrics
        train_loss = sum(
            pl_module.train_prop['loss']) / len(pl_module.train_prop['loss'])
        train_acc100 = sum(
            pl_module.train_prop['HR@100']) / len(pl_module.train_prop['HR@100'])
        # train_acc20 = sum(
        #     pl_module.train_prop['HR@20']) / len(pl_module.train_prop['HR@20'])
        # train_acc1 = sum(
        #     pl_module.train_prop['HR@1']) / len(pl_module.train_prop['HR@1'])
        test_loss = sum(
            pl_module.test_prop['loss']) / len(pl_module.test_prop['loss'])
        
        test_acc100 = sum(pl_module.test_prop['HR@100']) / \
            len(pl_module.test_prop['HR@100'])
        # test_acc20 = sum(pl_module.test_prop['HR@20']) / \
        #     len(pl_module.test_prop['HR@20'])
        # test_acc1 = sum(pl_module.test_prop['HR@1']) / \
        #     len(pl_module.test_prop['HR@1'])

        # Log the metrics
        pl_module.log(name='Loss on train', value=train_loss,
                      on_epoch=True, prog_bar=True, logger=True)
        pl_module.log(name='Loss on test', value=test_loss,
                      on_epoch=True, prog_bar=True, logger=True)

        pl_module.log(name='HR@100 on train', value=train_acc100,
                      on_epoch=True, prog_bar=True, logger=True)
        pl_module.log(name='HR@100 on test', value=test_acc100,
                            on_epoch=True, prog_bar=True, logger=True)
        
        # pl_module.log(name='HR@20 on train', value=train_acc20,
        #               on_epoch=True, prog_bar=True, logger=True)
        # pl_module.log(name='HR@20 on test', value=test_acc20,
        #               on_epoch=True, prog_bar=True, logger=True)
        
        
        # pl_module.log(name='HR@1 on train', value=train_acc1,
        #               on_epoch=True, prog_bar=True, logger=True)
        # pl_module.log(name='HR@1 on test', value=test_acc1,
        #               on_epoch=True, prog_bar=True, logger=True)

        
        
        
        

        # Re-initialize the metrics
        pl_module.train_prop['loss'] = []
        pl_module.train_prop['HR@100'] = []
        pl_module.train_prop['HR@20'] = []
        pl_module.train_prop['HR@1'] = []

        pl_module.test_prop['loss'] = []
        pl_module.test_prop['HR@100'] = []
        pl_module.test_prop['HR@20'] = []
        pl_module.test_prop['HR@1'] = []


## PyTorch Lightning Training Module (Node Classification)

In [10]:
class TrainingModule(pl.LightningModule):

    def __init__(self, model, predictor, lr, wd):
        super().__init__()
        self.model = model.to(device)
        self.predictor = predictor.to(device)
        self.lr = lr
        self.wd = wd

        self.train_prop = {'loss': [], 'HR@100': [], 'HR@20': [], 'HR@1': []}
        self.test_prop = {'loss': [], 'HR@100': [], 'HR@20': [], 'HR@1': []}

    def training_step(self, batch, batch_idx):

        out = self.model(batch)

        pos_edge = batch.pos_edge_label_index

        pos_pred = self.predictor(
            out[pos_edge[0]], out[pos_edge[1]], training=True)

        neg_edge = batch.neg_edge_label_index

        neg_pred = self.predictor(
            out[neg_edge[0]], out[neg_edge[1]], training=True)

        loss = -torch.log(pos_pred + 1e-15).mean() - \
            torch.log(1 - neg_pred[:pos_pred.shape[0]] + 1e-15).mean()

        acc100 = evaluate(pos_pred, neg_pred[pos_pred.shape[0]: 2*pos_pred.shape[0]], k=100)
        # acc20 = evaluate(pos_pred, neg_pred, k = 20)
        # acc1 = evaluate(pos_pred, neg_pred, k = 1)

        self.train_prop['loss'].append(loss)
        self.train_prop['HR@100'].append(acc100)
        # self.train_prop['HR@20'].append(acc20)
        # self.train_prop['HR@1'].append(acc1)

        return loss

    def validation_step(self, batch, batch_idx):

        out = self.model(batch)

        pos_edge = batch.pos_edge_label_index

        # training is for dropout
        pos_pred = self.predictor(
            out[pos_edge[0]], out[pos_edge[1]], training=False)

        neg_edge = batch.neg_edge_label_index

        # training is for dropout
        neg_pred = self.predictor(
            out[neg_edge[0]], out[neg_edge[1]], training=False)

        loss = -torch.log(pos_pred + 1e-15).mean() - \
            torch.log(1 - neg_pred[:pos_pred.shape[0]] + 1e-15).mean()

        acc100 = evaluate(pos_pred, neg_pred[pos_pred.shape[0]: pos_pred.shape[0]*2], k=100)
        # acc20 = evaluate(pos_pred, neg_pred, k = 20)
        # acc1 = evaluate(pos_pred, neg_pred, k = 1)

        self.test_prop['loss'].append(loss)
        self.test_prop['HR@100'].append(acc100)
        # self.test_prop['HR@20'].append(acc20)
        # self.test_prop['HR@1'].append(acc1)

        return loss

    def configure_optimizers(self):
        self.optimizer = torch.optim.Adam(
            list(self.model.parameters()) + list(self.predictor.parameters()), lr=self.lr, weight_decay=self.wd)
        return self.optimizer


def evaluate(pos_pred, neg_pred, k=100):
    n_indices = pos_pred.shape[0]
    hr = 0

    k = min(neg_pred.shape[0]+1, k)

    for pos_idx in range(n_indices):
        pos = pos_pred[pos_idx].unsqueeze(0)

        # Checking if the predictions are the same over all the negative distribution
        if round(pos.item(), 4) == round(torch.mean(neg_pred).item(), 4) and round(pos.item(), 4) == round(torch.min(neg_pred).item(), 4) and \
                round(pos.item(), 4) == round(torch.max(neg_pred).item(), 4):
            continue
        tot_tensor = torch.cat((neg_pred, pos), dim=0)
        scores_idx = torch.topk(tot_tensor.squeeze(1), k).indices

        # Check if the positive is in the top100. Positive is marked by the neg_pred.shape[0]
        if neg_pred.shape[0] in scores_idx:

            hr += 1
    return hr/n_indices

In [11]:

# hp enables a grid search on a wide set of hyperparameters.
if not sweep or mode == 'test':
    model = PhysicsGNN_LP(dataset, hidden_dim, num_layers, step = step)
    # model = GNNStack(dataset.x.shape[1], hidden_dim, hidden_dim, num_layers, dropout, emb=True)
    predictor = LinkPredictor(
        hidden_dim, output_dim, mlp_layer, link_bias, dropout, device=device)
    # predictor = LinkPredictor(
    #      hidden_dim, hidden_dim, 1, num_layers,
    #              dropout)
    
    pl_training_module = TrainingModule(model, predictor, lr, wd)

### Hyperparameters Tuning

In [ ]:
def sweep_train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        model = PhysicsGNN_LP(dataset, config.hidden_dim,
                              config.num_layers, step=config.step)
        predictor = LinkPredictor(
            config.hidden_dim, config.output_dim, config.mlp_layer, config.link_bias, config.dropout, device=device)
        pl_training_module = TrainingModule(
            model, predictor, config.lr, config.wd)
        exp_name = "Sweep_LinkPred"
        wandb_logger = WandbLogger(
            project=project_name, name=exp_name, config=hyperparameters)
        trainer = trainer = pl.Trainer(
            max_epochs=epochs,  # maximum number of epochs.
            gpus=num_gpus,  # the number of gpus we have at our disposal.
            default_root_dir="", callbacks=[Get_Metrics(), EarlyStopping('Loss on test', mode='min', patience=15)],
            logger=wandb_logger
        )
        trainer.fit(model=pl_training_module, datamodule=dataM)


if mode == 'hp' and sweep:

    import pprint

    pprint.pprint(sweep_config)

    sweep_id = wandb.sweep(sweep_config, project=project_name)

    wandb.agent(sweep_id, sweep_train, count=500)

    wandb.finish()

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'HR@100 on test'},
 'parameters': {'dropout': {'values': [0, 0.2, 0.3, 0.4]},
                'hidden_dim': {'values': [32, 64, 128, 256]},
                'link_bias': {'values': [True, False]},
                'lr': {'values': [0.01, 0.001, 0.0001]},
                'mlp_layer': {'values': [0, 1, 2, 3]},
                'num_layers': {'values': [1, 2, 3]},
                'output_dim': {'values': [16, 32, 64]},
                'step': {'values': [0.1, 0.2, 0.3]},
                'wd': {'values': [0, 0.01, 0.001, 1e-06]}}}
Create sweep with ID: t2dgzrq6
Sweep URL: https://wandb.ai/deepl_wizards/Link%20Prediction%20with%20PBGNN/sweeps/t2dgzrq6


wandb: Agent Starting Run: iiwvnop7 with config:
wandb: 	dropout: 0.2
wandb: 	hidden_dim: 256
wandb: 	link_bias: True
wandb: 	lr: 0.001
wandb: 	mlp_layer: 0
wandb: 	num_layers: 1
wandb: 	output_dim: 64
wandb: 	step: 0.1
wandb: 	wd: 0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type          | Params
--------------------------------------------
0 | model     | PhysicsGNN_LP | 433 K 
1 | predictor | LinkPredictor | 257   
--------------------------------------------
433 K     Trainable params
0         Non-trainable params
433 K     Total params
1.734     Total estimated model params size (MB)
/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /home/peppe/Desktop/Università/Projects/Link_Prediction_with_

Validation sanity check:   0%|          | 0/1 [00:00<?, ?it/s]

/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

HR@100 on test,▁▁▂▃▃▃▃▃▄▄▄▅▅▆▆▆▆▇▇█▇▇▇▇▇▇██▇▇▇▇▇▇▇▇▇▇▇▇
HR@100 on train,▁▂▂▁▂▄▄▄▃▄▅▅▆▅▆▅▅▇▆▆▆▆▇▆▇▇▇▇▇▆█▇██▇▇▇▇██
Loss on test,██████████████████████▇▇▇▇▇▆▆▆▅▅▄▃▃▂▂▁▁▁
Loss on train,████████████████████████▇▇▇▇▇▇▆▆▅▅▄▄▃▃▂▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
HR@100 on test,0.39658
HR@100 on train,0.11506
Loss on test,1.35313
Loss on train,1.33864
epoch,165


Epoch 171:  50%|█████     | 1/2 [00:00<00:00,  2.24it/s, loss=1.34, v_num=4wcx, Loss on train=1.340, Loss on test=1.350, HR@100 on train=0.122, HR@100 on test=0.398] 

Exception in thread SockSrvRdThr:
Traceback (most recent call last):
  File "/home/peppe/anaconda3/envs/my_env/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/wandb/sdk/service/server_sock.py", line 112, in run
    shandler(sreq)
  File "/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/wandb/sdk/service/server_sock.py", line 173, in server_record_publish
    iface = self._mux.get_stream(stream_id).interface
  File "/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/wandb/sdk/service/streams.py", line 199, in get_stream
    stream = self._streams[stream_id]
KeyError: '6w5r4wcx'


Epoch 177: 100%|██████████| 2/2 [00:00<00:00,  4.11it/s, loss=1.34, v_num=4wcx, Loss on train=1.330, Loss on test=1.360, HR@100 on train=0.113, HR@100 on test=0.395]


/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  rank_zero_deprecation(


In [33]:
if wb:
    exp_name = "Node_class_lr: " + \
        str(hyperparameters['learning rate']) + \
        '_wd: ' + str(hyperparameters['weight decay'])
    description = ' initial tests'
    exp_name += description
    wandb_logger = WandbLogger(
        project=project_name, name=exp_name, config=hyperparameters)


trainer = trainer = pl.Trainer(
    max_epochs=epochs,  # maximum number of epochs.
    gpus=num_gpus,  # the number of gpus we have at our disposal.
    default_root_dir="", callbacks=[Get_Metrics(), EarlyStopping('Loss on test', mode='min', patience=15)],
    logger=wandb_logger if wb else None

)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [34]:
trainer.fit(model = pl_training_module, datamodule = dataM)
if wb:
    wandb.finish()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type          | Params
--------------------------------------------
0 | model     | PhysicsGNN_LP | 200 K 
1 | predictor | LinkPredictor | 128   
--------------------------------------------
200 K     Trainable params
0         Non-trainable params
200 K     Total params
0.801     Total estimated model params size (MB)


Epoch 37: 100%|██████████| 2/2 [00:00<00:00,  4.54it/s, loss=0.544, Loss on train=0.440, Loss on test=1.380, HR@100 on train=0.384, HR@100 on test=0.755]


In [38]:
data = dataM.test_set
out = pl_training_module.model(data)
neg_shape = data.pos_edge_label_index.shape[1]
print(neg_shape)
preds = pl_training_module.predictor(out[data.pos_edge_label_index[0]], out[data.pos_edge_label_index[1]], training = False)
preds_neg = pl_training_module.predictor(out[data.neg_edge_label_index[0][:neg_shape]], out[data.neg_edge_label_index[1][:neg_shape]], training = False)


527


In [39]:
print(torch.mean(preds_neg))
pos = torch.tensor([[torch.mean(preds).item()]])
print(pos)

tensor(0.2873, grad_fn=<MeanBackward0>)
tensor([[0.7865]])


In [41]:
tot = torch.cat((preds_neg, pos), dim = 0)
top = torch.topk(tot.squeeze(1), 100)
print(top)

if preds_neg.shape[0] in top.indices:
    print("inside")

torch.return_types.topk(
values=tensor([1.0000, 1.0000, 1.0000, 0.9999, 0.9996, 0.9996, 0.9993, 0.9992, 0.9991,
        0.9991, 0.9990, 0.9989, 0.9988, 0.9985, 0.9977, 0.9969, 0.9969, 0.9967,
        0.9958, 0.9956, 0.9944, 0.9935, 0.9904, 0.9892, 0.9884, 0.9875, 0.9867,
        0.9857, 0.9855, 0.9826, 0.9825, 0.9815, 0.9780, 0.9779, 0.9772, 0.9759,
        0.9739, 0.9711, 0.9710, 0.9702, 0.9699, 0.9653, 0.9644, 0.9642, 0.9608,
        0.9532, 0.9526, 0.9515, 0.9508, 0.9498, 0.9491, 0.9486, 0.9425, 0.9412,
        0.9387, 0.9367, 0.9351, 0.9343, 0.9302, 0.9298, 0.9218, 0.9215, 0.9187,
        0.9106, 0.9057, 0.9031, 0.8982, 0.8964, 0.8930, 0.8911, 0.8821, 0.8799,
        0.8798, 0.8776, 0.8722, 0.8697, 0.8686, 0.8681, 0.8679, 0.8497, 0.8402,
        0.8359, 0.8336, 0.8321, 0.8287, 0.8279, 0.8212, 0.8202, 0.8116, 0.8090,
        0.8065, 0.8041, 0.7950, 0.7949, 0.7942, 0.7865, 0.7862, 0.7783, 0.7764,
        0.7757], grad_fn=<TopkBackward0>),
indices=tensor([432, 491, 210, 248, 391, 111,

In [42]:
evaluate(preds, preds_neg, k = 100)

0.7058823529411765